In [2]:
!pip install cmake
!pip install dlib


  Using cached dlib-19.24.4.tar.gz (3.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [10 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Traceback (most recent call last):
        File "/home/walmache/MEGA/autoCapacitacion/Maestria-IA/1.percepcionComputacional/repos/practicasPercepcionComputacional/.venv/bin/cmake", line 7, in <module>
          from cmake import cmake
      ModuleNotFoundError: No module named 'cmake'
      
      ERROR: CMake must be installed to build dlib
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for dlib
Failed to build dlib
ERROR: Could not build wheels for dlib, which is required to inst

In [ ]:
import cv2  # Biblioteca para el procesamiento de imágenes en tiempo real
import face_recognition  # Biblioteca para el reconocimiento facial
import os  

In [ ]:
# Directorio donde se almacenan las imágenes de los rostros
rostros_dir = 'rostros/'

# Listas para almacenar las codificaciones de los rostros y sus nombres
known_face_encodings = []
known_face_names = []

# Recorrer el directorio y procesar cada imagen
for filename in os.listdir(rostros_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Cargar la imagen
        image_path = os.path.join(rostros_dir, filename)
        image = face_recognition.load_image_file(image_path)
        
        # Codificar el rostro
        face_encoding = face_recognition.face_encodings(image)[0]
        
        # Agregar la codificación y el nombre a las listas
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])  # Usar el nombre del archivo sin la extensión como nombre del rostro


In [ ]:
# Iniciar la captura de video desde la cámara
video_capture = cv2.VideoCapture(0)

while True:
    # Capturar un solo cuadro de video
    ret, frame = video_capture.read()
    
    # Reducir el tamaño del cuadro para acelerar el procesamiento
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Convertir el cuadro de BGR a RGB
    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Encontrar todos los rostros y sus codificaciones en el cuadro actual
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    # Inicializar una lista para los nombres de los rostros reconocidos
    face_names = []
    
    # Comparar cada codificación de rostro con las codificaciones conocidas
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        
        # Utilizar el rostro conocido con la menor distancia de codificación
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = face_distances.argmin()
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        
        face_names.append(name)
    
    # Mostrar los resultados
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Escalar las ubicaciones del rostro a partir del tamaño reducido
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Dibujar un cuadro alrededor del rostro
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        
        # Dibujar una etiqueta con el nombre debajo del rostro
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    
    # Mostrar la imagen resultante
    cv2.imshow('Video', frame)
    
    # Salir del bucle cuando se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar el manejador de la cámara y cerrar las ventanas
video_capture.release()
cv2.destroyAllWindows()
